In [451]:
import matplotlib.pyplot as plt
import numpy as np

In [452]:
np.set_printoptions(precision=8)

In [453]:
path="/Users/ecasiano/Desktop/PrototypeScripts/TestSystematicError/"
path="/Users/ecasiano/Xcode/pimc/pimc/"

In [454]:
seed = 42

In [455]:
data_SWAPn_m2 = np.loadtxt(path+"1D_10_10_5_3.300000_1.000000_2.000000_1000_1000_SWAPn-mA5_"+str(seed)+"_square.dat")
data_PnSquared_m2 = np.loadtxt(path+"1D_10_10_5_3.300000_1.000000_2.000000_1000_1000_PnSquared-mA5_"+str(seed)+"_square.dat")

data_SWAPn_m2.shape,data_PnSquared_m2.shape

((1000, 11), (1000, 11))

In [456]:
N=data_SWAPn_m2.shape[1]-1 # Get N

In [457]:
S2 = np.zeros(data_SWAPn_m2.shape)
SWAPn_m2_col_sum = np.sum(data_SWAPn_m2,axis=0)
PnSquared_m2_col_sum = np.sum(data_PnSquared_m2,axis=0)
for i in range(data_SWAPn_m2.shape[0]):
    PnSquared_m2_col_sum_i = PnSquared_m2_col_sum - data_PnSquared_m2[i]
    
    SWAPn_m2_col_sum_i = SWAPn_m2_col_sum - data_SWAPn_m2[i]
    
    S2[i] = -np.log(SWAPn_m2_col_sum_i/PnSquared_m2_col_sum_i)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


In [458]:
S2_mean = np.mean(S2,axis=0)
S2_mean

array([       nan,        nan,        nan, 0.06643753, 0.0467273 ,
       0.04220156, 0.05531334, 0.14744435,        nan,        nan,
              nan])

In [459]:
N_rows = S2.shape[0]

In [460]:
S2_err = np.std(S2,axis=0)
S2_err *= np.sqrt(N_rows-1)

In [461]:
for n in range(N+1):
    print(f"<S2(n={n})> = {S2_mean[n]:0.6f} +/- {S2_err[n]:0.6f}")

<S2(n=0)> = nan +/- nan
<S2(n=1)> = nan +/- nan
<S2(n=2)> = nan +/- nan
<S2(n=3)> = 0.066438 +/- 0.075838
<S2(n=4)> = 0.046727 +/- 0.011163
<S2(n=5)> = 0.042202 +/- 0.006056
<S2(n=6)> = 0.055313 +/- 0.012286
<S2(n=7)> = 0.147444 +/- 0.083591
<S2(n=8)> = nan +/- nan
<S2(n=9)> = nan +/- nan
<S2(n=10)> = nan +/- nan


In [462]:
data_Pn_m2 = np.loadtxt(path+"1D_10_10_5_3.300000_1.000000_2.000000_1000_1000_Pn-mA5_"+str(seed)+"_square.dat")
data_Pn_m2.shape

(1000, 11)

In [463]:
data_Pn_col_sum_m2 = np.sum(data_Pn_m2,axis=0)

In [464]:
jacknifed_Pn = np.zeros(data_Pn_m2.shape)
for i in range(data_Pn_m2.shape[0]):
    data_Pn_col_sum_m2_i = data_Pn_col_sum_m2 - data_Pn_m2[i]
        
    jacknifed_Pn[i] = data_Pn_col_sum_m2_i
jacknifed_Pn /= np.sum(jacknifed_Pn,axis=1)[:,None]

In [465]:
Pn_mean = np.mean(jacknifed_Pn,axis=0)
Pn_mean /= np.sum(Pn_mean )
N_rows_Pn = jacknifed_Pn.shape[0]
Pn_err = np.std(jacknifed_Pn,axis=0)
Pn_err *= np.sqrt(N_rows_Pn-1)

In [466]:
for n in range(N+1):
    print(f'P(n={n}) = {Pn_mean[n]:0.4f} +/- {Pn_err[n]:0.4f}')

P(n=0) = 0.0000 +/- 0.0000
P(n=1) = 0.0000 +/- 0.0000
P(n=2) = 0.0012 +/- 0.0001
P(n=3) = 0.0312 +/- 0.0002
P(n=4) = 0.2299 +/- 0.0006
P(n=5) = 0.4743 +/- 0.0007
P(n=6) = 0.2312 +/- 0.0006
P(n=7) = 0.0311 +/- 0.0003
P(n=8) = 0.0011 +/- 0.0000
P(n=9) = 0.0000 +/- 0.0000
P(n=10) = 0.0000 +/- 0.0000


# 